In [1]:
from arcgis.features import *
from arcgis.gis import GIS

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(color_codes=True)
%matplotlib inline

In [2]:
#User defined inputs
initiative_id = 'b04ef2c7a77a43218b3e70dd75cc4f5e'
indicator_id = 'crime'

### Connect to your Hub and fetch indicator

In [3]:
gis = GIS("https://dc.mapsqa.arcgis.com", "dcadminqa", "dcadminqa1")

In [4]:
myHub = gis.hub

In [5]:
data_inventory = myHub.initiatives.get(initiative_id)

In [6]:
crime_indicator = data_inventory.indicators.get(indicator_id)
crime_itemid = crime_indicator.itemid

In [7]:
crime_item = gis.content.get(crime_itemid)
crime_flayer = crime_item.layers[0]

# create a Spatially Enabled DataFrame object
crime = pd.DataFrame.spatial.from_layer(crime_flayer)

Exception: Could not load the dataset: [WinError 10054] An existing connection was forcibly closed by the remote host

### Total crime incidents in 2017

In [ ]:
print(crime.shape[0])

### Bar chart for frequency of crime based on crime type

In [ ]:
crime_indicator.mappings

In [ ]:
crimeType_columnName = next(field['name'] for field in crime_indicator.mappings if field['id'] == 'crimeType')
shift_columnName = next(field['name'] for field in crime_indicator.mappings if field['id'] == 'shift')
date_columnName = next(field['name'] for field in crime_indicator.mappings if field['id'] == 'reported')

In [ ]:
#Frequency calculation of each crime type
counts = crime[crimeType_columnName].value_counts()
print(counts)

In [ ]:
#Generates bar graph
ax = counts.plot(kind='barh', figsize=(10, 10), legend=True, fontsize=12, color='green')

#X axis text and display style of categories
ax.set_xlabel("Count", fontsize=12)
#plt.xticks(rotation=45)

#Y axis text
ax.set_ylabel("Offense Type", fontsize=12)

#Title
ax.set_title("Types of crime based on frequency of occurence", fontsize=20)

### Pie chart for day/night crime

In [ ]:
#Getting unique values for shift
types = list(crime[shift_columnName].unique())
print(types)

In [ ]:
#Frequency calculation of each shift type
sizes = crime['SHIFT'].value_counts()
print(sizes)

In [ ]:
# Data to plot
labels = types
colors = ['lightskyblue', 'yellowgreen', 'lightcoral']
explode = (0.05, 0.015, 0)  # explode 1st slice
 
# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.2f%%', shadow=True, startangle=100)
 
plt.axis('equal')
plt.show()

### Pie chart for weekday (Mon-Thurs), weekend (Fri-Sun)

In [ ]:
import datetime
import pandas as pd

In [ ]:
def format_date(date):
    '''Return date in Y-M-D'''
    epoch_time = str(date)[0: 10]
    return epoch_time

In [ ]:
crime['date'] = crime[date_columnName].apply(format_date)
crime['date'] = pd.to_datetime(crime['date']).dt.date
crime['date'].head()

In [ ]:
crime['day_of_week'] = crime['date'].apply(lambda x: x.weekday()) 

In [ ]:
crime['day_of_week'].head()

In [ ]:
def week_day(num):
    '''Return Weekday/Weekend'''
    if num < 4:
        return 'Weekday'
    if num >= 4:
        return 'Weekend'

In [ ]:
crime['day'] = crime['day_of_week'].apply(week_day)

In [ ]:
#Getting unique values for day
types = list(crime['day'].unique())
print(types)

In [ ]:
#Frequency calculation of each day type
sizes = crime['day'].value_counts()
print(sizes)

In [ ]:
# Data to plot
labels = types
colors = ['yellow', 'orange']
explode = (0.05, 0.015)  # explode 1st slice
 
# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.2f%%', shadow=True, startangle=100)
 
plt.axis('equal')
plt.show()

### Histogram for crime frequency based on months

In [ ]:
def month(date):
    '''Return month number'''
    return str(date)[5:7]

In [ ]:
crime['month'] = crime['date'].apply(month)
crime['month'].head()

In [ ]:
crime_month = crime['month'].astype(int)
type(crime_month)

In [ ]:
plt.figure(figsize=(8,8))
n, bins, patches = plt.hist(crime_month, bins=[1,2,3,4,5,6,7,8,9,10,11,12], facecolor='blue', alpha=0.5)

plt.title("Number of crime incidents for every month of 2017", fontsize=16)
plt.xlabel("Month (January - December)", fontsize=16)
plt.ylabel("Frequency", fontsize=16)